In [3]:
%pip install -U git+https://github.com/albumentations-team/albumentations

%pip install -U scikit-image

  Cloning https://github.com/albumentations-team/albumentations to c:\users\feers\appdata\local\temp\pip-req-build-31hid_s2
  Resolved https://github.com/albumentations-team/albumentations to commit 3e09e4bab7f582c7dce3473826558826f4d5df0e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached albucore-0.0.21-py3-none-any.whl.metadata (5.3 kB)
  Using cached eval_type_backport-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached albucore-0.0.21-py3-none-any.whl (12 kB)
Using cach

  Running command git clone --filter=blob:none --quiet https://github.com/albumentations-team/albumentations 'C:\Users\feers\AppData\Local\Temp\pip-req-build-31hid_s2'
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\feers\\anaconda3\\envs\\bacteri\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



  Using cached scikit_image-0.24.0-cp39-cp39-win_amd64.whl.metadata (14 kB)
  Using cached imageio-2.36.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached tifffile-2024.8.30-py3-none-any.whl.metadata (31 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached scikit_image-0.24.0-cp39-cp39-win_amd64.whl (12.9 MB)
Using cached imageio-2.36.0-py3-none-any.whl (315 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
Using cached tifffile-2024.8.30-py3-none-any.whl (227 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import random
import shutil
import albumentations as A
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor, as_completed
from skimage.io import imread
from threading import Lock

# Path ke folder data
images_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7"
labels_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\EMDS7xml"
output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\\temp"
sample_output_dir = "D:\KULIAH\SEMESTER 5\RSBP\Bacteri\Detection-of-bacteria-in-water-using-Yolo-images\Detect\Detection-of-bacteria-in-water-using-Yolo-images\sample_output"

train_val_split = 0.8  # Rasio data train/validation

# Cek apakah path gambar dan label ada
if not os.path.exists(images_dir):
    raise FileNotFoundError(f"Path gambar tidak ditemukan: {images_dir}")
if not os.path.exists(labels_dir):
    raise FileNotFoundError(f"Path label tidak ditemukan: {labels_dir}")

# Remove output directory if it exists and create fresh output folders
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

if os.path.exists(sample_output_dir):
    shutil.rmtree(sample_output_dir)

# Buat ulang folder output yang fresh
os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
os.makedirs(sample_output_dir, exist_ok=True)

# Daftar semua gambar dan annotations
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
label_files = [f.replace('.png', '.xml') for f in image_files]

# Shuffle data for randomness
data = list(zip(image_files, label_files))
random.shuffle(data)

# Split data into training and validation sets
split_index = int(train_val_split * len(data))
train_data = data[:split_index]
val_data = data[split_index:]

class_mapping = {
    'G001': 0, 'G002': 1, 'G003': 2, 'G004': 3, 'G005': 4, 'G006': 5, 'G007': 6, 'G008': 7, 'G009': 8, 'G010': 9,
    'G011': 10, 'G012': 11, 'G013': 12, 'G014': 13, 'G015': 14, 'G016': 15, 'G017': 16, 'G018': 17, 'G019': 18, 'G020': 19,
    'G021': 20, 'G022': 21, 'G023': 22, 'G024': 23, 'G025': 24, 'G026': 25, 'G027': 26, 'G028': 27, 'G029': 28, 'G030': 29,
    'G031': 30, 'G032': 31, 'G033': 32, 'G034': 33, 'G035': 34, 'G036': 35, 'G037': 36, 'G038': 37, 'G039': 38, 'G040': 39,
    'G041': 40, 'G042': 41, 'G043': 42, 'G044': 43, 'G045': 44, 'G046': 45, 'G047': 46, 'G048': 47, 'G049': 48, 'G050': 49,
    'G051': 50, 'G052': 51, 'G053': 52
}

# Augmentasi gambar dan bounding box tanpa mirroring, mengisi area kosong dengan putih
augmentor = A.Compose([
    A.Rotate(limit=30, p=0.3),
    A.RandomBrightnessContrast(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Resize(512, 512, p=1.0)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

sample_counter_lock = Lock()

def draw_bboxes(image, bboxes, class_labels):
    """Gambar bounding box pada gambar."""
    for bbox, class_id in zip(bboxes, class_labels):
        x_center, y_center, width, height = bbox
        img_h, img_w = image.shape[:2]
        xmin = int((x_center - width / 2) * img_w)
        ymin = int((y_center - height / 2) * img_h)
        xmax = int((x_center + width / 2) * img_w)
        ymax = int((y_center + height / 2) * img_h)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

def convert_xml_to_txt(xml_path, txt_output_path, augment=False, img=None):
    """Fungsi ini akan mengonversi format XML ke format YOLO (txt)."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    bboxes = []
    class_labels = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = class_mapping.get(class_name, -1)
        if class_id == -1:
            continue
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        x_center = (xmin + xmax) / 2.0 / width
        y_center = (ymin + ymax) / 2.0 / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height
        bboxes.append([x_center, y_center, box_width, box_height])
        class_labels.append(class_id)

    with open(txt_output_path, 'w') as txt_file:
        for bbox, label in zip(bboxes, class_labels):
            txt_file.write(f"{label} {' '.join(map(str, bbox))}\n")

    return img, bboxes, class_labels

def apply_morphology(img, kernel_size=(10, 10), blur_kernel=(5, 5)):
    """Terapkan operasi thresholding dengan Gaussian blur, median blur, top-hat, dan adaptive thresholding."""
    # Jika gambar berwarna, konversi ke grayscale
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # Tambahkan Gaussian blur untuk mengurangi noise awal
    blurred = cv2.GaussianBlur(gray, blur_kernel, 0)
    
    # Membuat elemen struktural berbentuk disk kecil untuk top-hat
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    
    # Top-hat untuk menonjolkan detail bakteri
    tophat_img = cv2.morphologyEx(blurred, cv2.MORPH_TOPHAT, kernel)
    
    # Tambahkan Median Blur pada hasil top-hat untuk mengurangi noise
    tophat_blur = cv2.medianBlur(tophat_img, 5)
    
    # Bilateral Filter untuk menghaluskan area dalam tanpa menghilangkan tepi
    tophat_bilateral = cv2.bilateralFilter(tophat_blur, d=9, sigmaColor=75, sigmaSpace=75)
    
    # Adaptive thresholding setelah top-hat dan blur
    bwnewimg = cv2.adaptiveThreshold(tophat_bilateral, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
    
    return bwnewimg

def process_data(img_file, lbl_file, img_dest_folder, lbl_dest_folder, augment=False, sample_output_count=15):
    img_src_path = os.path.join(images_dir, img_file)
    lbl_src_path = os.path.join(labels_dir, lbl_file)

    # Baca gambar dan konversi ke grayscale
    img = imread(img_src_path)
    img = apply_morphology(img)
    
    if augment:
        img_dest_path = os.path.join(img_dest_folder, img_file.replace('.png', '_aug.png'))
        lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '_aug.txt'))
        augmented_img, augmented_bboxes, class_labels = convert_xml_to_txt(lbl_src_path, lbl_dest_path, augment=True, img=img)

        augmented = augmentor(image=augmented_img, bboxes=augmented_bboxes, class_labels=class_labels)
        augmented_img = augmented['image']
        augmented_bboxes = augmented['bboxes']

        # Draw bounding boxes on augmented image and save to sample output
        with sample_counter_lock:
            if sample_output_count > 0:
                sample_img = augmented_img.copy()
                draw_bboxes(sample_img, augmented_bboxes, class_labels)
                cv2.imwrite(os.path.join(sample_output_dir, f"{img_file.replace('.png', '_aug_sample.png')}") , sample_img)
                sample_output_count -= 1

        # Save the augmented image in the train directory
        cv2.imwrite(img_dest_path, augmented_img)
        print(f"Augmented Image saved to: {img_dest_path}")
    else:
        img_dest_path = os.path.join(img_dest_folder, img_file)
        lbl_dest_path = os.path.join(lbl_dest_folder, lbl_file.replace('.xml', '.txt'))
        shutil.copy(img_src_path, img_dest_path)
        convert_xml_to_txt(lbl_src_path, lbl_dest_path)
        print(f"Original Image saved to: {img_dest_path}")

def copy_and_convert_data(data, img_dest_folder, lbl_dest_folder, augment=False, sample_output_count=30):
    with ThreadPoolExecutor() as executor:
        futures = []
        for img_file, lbl_file in data:
            futures.append(executor.submit(process_data, img_file, lbl_file, img_dest_folder, lbl_dest_folder, augment, sample_output_count))
        for future in as_completed(futures):
            future.result()

copy_and_convert_data(train_data, os.path.join(output_dir, 'images/train'), os.path.join(output_dir, 'labels/train'), augment=True)
copy_and_convert_data(val_data, os.path.join(output_dir, 'images/val'), os.path.join(output_dir, 'labels/val'))

print("Data berhasil dipisah, dikonversi, dan diaugmentasi ke format YOLO! Contoh gambar disimpan di folder sample_output.")